# Doc2Vec model
> * Positive or Negative

In [1]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re

from collections import namedtuple
from gensim.models import doc2vec
import multiprocessing
from konlpy.tag import Mecab
from konlpy.utils import pprint
import numpy as np
from ckonlpy.tag import Twitter as ctwitter
mecab = Mecab()
ct = ctwitter()

In [2]:
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
from sklearn.model_selection import train_test_split

In [3]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags')

# Doc2Vec 생성

* vector size
 [ 1000, 2000 ]

In [ ]:
rawdata = pd.read_csv('./data/sentiment_data/raw_data_for_sentiment.txt',header=None,encoding='utf-8')
print (rawdata.shape)

### tagging twitter

In [ ]:
# twitter
def tokenize1(doc):
    return ['/'.join(t) for t in ct.pos(doc)]

In [ ]:
raw_doc_ct = [(tokenize1(rawdata.loc[idx][0]), rawdata.loc[idx][1]) for idx in rawdata.index]
pickle.dump(raw_doc_ct, open('./data/pre_data/pre_data_by_ct_for_sentiment_analysis.pickled','wb'))

In [ ]:
tagged_ct = [TaggedDocument(d, [c]) for d, c in raw_doc_ct]
pickle.dump(tagged_ct, open('./data/pre_data/pre_by_ct_data_tagged_run_docs.pickled','wb'))

#### model 만들기

In [ ]:
tagged_ct = pickle.load(open('./data/pre_data/pre_by_ct_data_tagged_run_docs.pickled','rb'))

In [ ]:
train, test = train_test_split(tagged_ct, test_size=0.15, random_state=42)

pickle.dump(train, open('./data/pre_data/pre_by_ct_train.pickled','wb'))
pickle.dump(test, open('./data/pre_data/pre_by_ct_test.pickled','wb'))

In [ ]:
train = pickle.load(open('./data/pre_data/pre_by_ct_train.pickled','rb'))
test = pickle.load(open('./data/pre_data/pre_by_ct_test.pickled','rb'))

In [ ]:
doc_vectorizer = doc2vec.Doc2Vec(size=1000, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer.build_vocab(train)

for epoch in range(20):
    doc_vectorizer.train(train, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.0015  # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha  # fix the learning rate, no decay

#To save
doc_vectorizer.save('./model/doc2vec_size1000_epoch30_by_ct.model')
pprint(doc_vectorizer.most_similar('문재인/Noun'))
pprint(doc_vectorizer.most_similar('노무현/Noun'))
pprint(doc_vectorizer.most_similar('박근혜/Noun'))

In [ ]:
doc_vectorizer = doc2vec.Doc2Vec(size=2000, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer.build_vocab(train)

for epoch in range(20):
    doc_vectorizer.train(train, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.0015  # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha  # fix the learning rate, no decay

#To save
doc_vectorizer.save('./model/doc2vec_size2000_epoch20_by_ct.model')
pprint(doc_vectorizer.most_similar('문재인/Noun'))
pprint(doc_vectorizer.most_similar('노무현/Noun'))
pprint(doc_vectorizer.most_similar('박근혜/Noun'))

* size 1000, epoch 20 으로 모델 생성
* size 2000, epoch 20 으로 모델 생성

In [ ]:
doc_vectorizer = doc2vec.Doc2Vec(size=2500, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer.build_vocab(train)

for epoch in range(20):
    doc_vectorizer.train(train, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.0015  # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha  # fix the learning rate, no decay

#To save
doc_vectorizer.save('./model/doc2vec_size2500_epoch20_by_ct.model')
pprint(doc_vectorizer.most_similar('문재인/Noun'))
pprint(doc_vectorizer.most_similar('노무현/Noun'))
pprint(doc_vectorizer.most_similar('박근혜/Noun'))

### tagging mecab

In [ ]:
# mecab
def tokenize2(doc):
    return ['/'.join(t) for t in mecab.pos(doc)]

In [ ]:
raw_doc_mecab = [(tokenize2(rawdata.loc[idx][0]), rawdata.loc[idx][1]) for idx in rawdata.index]
pickle.dump(raw_doc_mecab, open('./data/pre_data/pre_data_by_mecab_for_sentiment_analysis.pickled','wb'))

In [ ]:
tagged_mecab = [TaggedDocument(d, [c]) for d, c in raw_doc_mecab]
pickle.dump(tagged_mecab, open('./data/pre_data/pre_by_mecab_data_tagged_run_docs.pickled','wb'))

#### model 만들기

In [16]:
tagged_mecab = pickle.load(open('./data/pre_data/pre_by_mecab_data_tagged_run_docs.pickled','rb'))

In [13]:
train2, test2 = train_test_split(tagged_mecab, test_size=0.15, random_state=42)

pickle.dump(train2, open('./data/pre_data/pre_by_mecab_train.pickled','wb'))
pickle.dump(test2, open('./data/pre_data/pre_by_mecab_test.pickled','wb'))

In [ ]:
train2 = pickle.load(open('./data/pre_data/pre_by_mecab_train.pickled','rb'))
test2 = pickle.load(open('./data/pre_data/pre_by_mecab_test.pickled','rb'))

In [ ]:
doc_vectorizer2 = doc2vec.Doc2Vec(size=1000, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer2.build_vocab(train2)

for epoch in range(20):
    doc_vectorizer2.train(train2, total_examples=doc_vectorizer2.corpus_count, epochs=doc_vectorizer2.iter)
    doc_vectorizer2.alpha -= 0.0015  # decrease the learning rate
    doc_vectorizer2.min_alpha = doc_vectorizer2.alpha  # fix the learning rate, no decay

#To save
doc_vectorizer2.save('./model/doc2vec_size1000_epoch20_by_mecab.model')
pprint(doc_vectorizer2.most_similar('문재인/NNP'))
pprint(doc_vectorizer2.most_similar('노무현/NNP'))
pprint(doc_vectorizer2.most_similar('박근혜/NNP'))

In [ ]:
doc_vectorizer2 = doc2vec.Doc2Vec(size=2000, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer2.build_vocab(train2)

for epoch in range(20):
    doc_vectorizer2.train(train2, total_examples=doc_vectorizer2.corpus_count, epochs=doc_vectorizer2.iter)
    doc_vectorizer2.alpha -= 0.0015  # decrease the learning rate
    doc_vectorizer2.min_alpha = doc_vectorizer2.alpha  # fix the learning rate, no decay

#To save
doc_vectorizer2.save('./model/doc2vec_size2000_epoch20_by_mecab.model')
pprint(doc_vectorizer2.most_similar('문재인/NNP'))
pprint(doc_vectorizer2.most_similar('노무현/NNP'))
pprint(doc_vectorizer2.most_similar('박근혜/NNP'))

* size 1000, epoch 20 으로 모델 생성
* size 2000, epoch 20 으로 모델 생성

In [ ]:
doc_vectorizer2 = doc2vec.Doc2Vec(size=2500, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer2.build_vocab(train2)

for epoch in range(20):
    doc_vectorizer2.train(train2, total_examples=doc_vectorizer2.corpus_count, epochs=doc_vectorizer2.iter)
    doc_vectorizer2.alpha -= 0.0015  # decrease the learning rate
    doc_vectorizer2.min_alpha = doc_vectorizer2.alpha  # fix the learning rate, no decay

#To save
doc_vectorizer2.save('./model/doc2vec_size2500_epoch20_by_mecab.model')
pprint(doc_vectorizer2.most_similar('문재인/NNP'))
pprint(doc_vectorizer2.most_similar('노무현/NNP'))
pprint(doc_vectorizer2.most_similar('박근혜/NNP'))